In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
# direct use real_label,normalize var as 0.5
fake_noise_normal = torch.from_numpy(np.random.normal(loc=0.5, scale=0.5, size=[2048,1,32,32])) # var is 1/4
fake_noise_normal = fake_noise_normal.type(torch.FloatTensor)
fake_noise_normal = fake_noise_normal.to(device)
fake_label = torch.from_numpy(np.random.randint(10,size=[2048]))

fake_noise_uniform = np.sqrt(3)*torch.from_numpy(np.random.uniform(0,1,[2048,1,32,32])) # 1/12
fake_noise_uniform = fake_noise_uniform.type(torch.FloatTensor)
fake_noise_uniform = fake_noise_uniform.to(device)

fake_noise_cauchy = torch.from_numpy(np.random.standard_cauchy(size=[2048,1,32,32])+0.5)
fake_noise_cauchy = fake_noise_cauchy.type(torch.FloatTensor)
fake_noise_cauchy = fake_noise_cauchy.to(device)

fake_noise_laplace = torch.from_numpy(np.random.laplace(loc=0.5, scale=1.0, size=[2048,1,32,32]))/ np.sqrt(8) # var is 2
fake_noise_laplace = fake_noise_laplace.type(torch.FloatTensor)
fake_noise_laplace = fake_noise_laplace.to(device)

fake_noise_gumbel = torch.from_numpy(np.random.gumbel(loc=0.5,scale=1.0,size=[2048,1,32,32]))/ np.sqrt(6.58) #pi^2/6 1.645
fake_noise_gumbel = fake_noise_gumbel.type(torch.FloatTensor)
fake_noise_gumbel = fake_noise_gumbel.to(device)
# from 0.01 - 0.99,0 -1
epsilon_noise = 0.1

In [4]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [5]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10



In [6]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius_noise(radius1,radius2,netD,noise, real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [7]:
#acc at first for NetD, there exist some issues for netG here, resample will get it right.
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.831
SVD: 0.8248
NetD, Acc at first 0.8387


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.8075
SVD: 0.7996
NetG, Acc at first 0.841


In [8]:
# data_preprocessing and parameters
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'mnist'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()

In [9]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 1
selected_radius = 0.7
#data_list = [real_cpu,fake_noise_normal,fake_noise_uniform,fake_noise_laplace,fake_noise_cauchy,fake_noise_gumbel, new_sample,new_sample2,new_sample3]
TT = 0
TF = 0
FT = 0
FF = 0
data_list = [real_cpu,real_cpu2,fake_noise_normal]
label_list = [real_label,real_label2,real_label2]
start_epoch = 20
for epoch in range(num_steps):
    if epoch < start_epoch:

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()

            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        
    else:
        print('epoch',epoch)
        pos = random.choice(range(3))
        radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=data_list[pos],real_label=label_list[pos],tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        print(radius)
        print(pos)
        if radius > selected_radius:
            print('This data is wrong to this model, wrong distribution,pass')
            if pos ==0:
                TF +=1
            else:
                FF +=1
            pass
        else:
            radius_list.append(radius)
            if pos == 0:
                TT += 1
            else:
                FT += 1
            for i in range(n_dis):
                netD.zero_grad()
                optD.zero_grad()
                Z = netD(data_list[pos])
                net_noise = get_noise(DATASET, device, LABEL)
                fed_state_dict = get_netG(netD,netV,net_noise,radius)
                netG.load_state_dict(fed_state_dict)
                Z_bar = netG(data_list[pos])
                errD, errD_EC = mcr_gan_loss(Z, Z_bar, label_list[pos], 1, n_dis)
                print(errD, errD_EC)
                errD.backward()
                optD.step()
            temp = 0
            for j in range(n_dis2):
                netG.zero_grad()
                optG.zero_grad()

                Z = netD(data_list[pos])
                net_noise = get_noise(DATASET, device, LABEL)
                fed_state_dict = get_netG(netD,netV,net_noise,radius)
                netG.load_state_dict(fed_state_dict)
                Z_bar = netG(data_list[pos])
                errG, errG_EC = mcr_gan_loss(Z, Z_bar, label_list[pos], n_dis2 - 1, n_dis2)
                print(errG, errG_EC)
                temp = errG
                errG = (-1) * errG
                errG.backward()
                optG.step()

                update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
                netV.load_state_dict(update_dic)
            


78.08533627012652
tensor(-14.9390, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.6596, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.5902, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.0118, device='cuda:0', grad_fn=<AddBackward0>)]
8.492080861065174
tensor(-13.3899, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.6847, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.0871, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.2662, device='cuda:0', grad_fn=<AddBackward0>)]
99.99063782035199
tensor(-14.3880, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.6847, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.0065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.3164, device='cuda:0', grad_fn=<AddBackward0>)]
1.4410384315598765
tensor(-20.0812, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.4072, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1919, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.5266, device='cuda:0', grad_fn=<AddBack

0.5310960943750249
tensor(-48.4029, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.4857, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.6457, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.6945, device='cuda:0', grad_fn=<AddBackward0>)]
0.5149327967465258
tensor(-48.1463, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.4857, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.4445, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1722, device='cuda:0', grad_fn=<AddBackward0>)]
0.4118577560139378
tensor(-47.8799, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.4129, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.5439, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.8419, device='cuda:0', grad_fn=<AddBackward0>)]
0.4602842919115599
tensor(-48.2113, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.4129, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.7679, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.6182, device='cuda:0', grad_fn=<

epoch 52
1.0010074695184807
1
This data is wrong to this model, wrong distribution,pass
epoch 53
1.147369598231177
1
This data is wrong to this model, wrong distribution,pass
epoch 54
1.0200351040455737
1
This data is wrong to this model, wrong distribution,pass
epoch 55
2.4068493799626642
2
This data is wrong to this model, wrong distribution,pass
epoch 56
2.3105685496036314
2
This data is wrong to this model, wrong distribution,pass
epoch 57
0.46924143234454274
0
tensor(-56.1957, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.5305, device='cuda:0', grad_fn=<NegBackward0>), tensor(-26.6153, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.7666, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-56.9954, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.0196, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.0649, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.7590, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 58
1.646389848192208
2
This data is wrong to this mode

0.4196875601851819
0
tensor(-59.3782, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.3598, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3167, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.4834, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-60.8851, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.8976, device='cuda:0', grad_fn=<NegBackward0>), tensor(-29.2806, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5125, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 90
1.0879652578836132
1
This data is wrong to this model, wrong distribution,pass
epoch 91
1.4980686230529017
2
This data is wrong to this model, wrong distribution,pass
epoch 92
1.1218794237485399
1
This data is wrong to this model, wrong distribution,pass
epoch 93
0.40982105404325175
0
tensor(-60.8177, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.8976, device='cuda:0', grad_fn=<NegBackward0>), tensor(-29.2033, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5669, device='cuda:0', grad

0.39551916187519487
0
tensor(-65.9101, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.5609, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.8300, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.5682, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.4331, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.7754, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.1292, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.6113, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 127
0.3867107675540861
0
tensor(-66.0910, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.7754, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.6327, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3813, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-65.8320, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.4876, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.8343, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.5268, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 1

...

epoch 342
1.344458688222149
1
This data is wrong to this model, wrong distribution,pass
epoch 343
1.8900272225979697
2
This data is wrong to this model, wrong distribution,pass
epoch 344
1.3166691438135645
1
This data is wrong to this model, wrong distribution,pass
epoch 345
1.318990723044232
1
This data is wrong to this model, wrong distribution,pass
epoch 346
0.3093478903938556
0
tensor(-81.2617, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-40.3347, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.8900, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.8210, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-80.9817, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-40.3622, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.5826, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.8206, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 347
0.31193489227317184
0
tensor(-81.1219, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-40.3622, device='cuda:0', grad_fn=<NegBackward

0.2933641654160088
0
tensor(-82.5128, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-41.0894, device='cuda:0', grad_fn=<NegBackward0>), tensor(-40.4954, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.5297, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-82.4395, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-41.0649, device='cuda:0', grad_fn=<NegBackward0>), tensor(-40.3913, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.6731, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 375
2.0621337366003556
2
This data is wrong to this model, wrong distribution,pass
epoch 376
1.999413709732441
2
This data is wrong to this model, wrong distribution,pass
epoch 377
1.7985253982110545
2
This data is wrong to this model, wrong distribution,pass
epoch 378
1.9606016467244756
2
This data is wrong to this model, wrong distribution,pass
epoch 379
1.1995439767338414
1
This data is wrong to this model, wrong distribution,pass
epoch 380
0.3173517893330054
0
tensor(-82.5989, device='cuda:0', 

0.32682920616583633
0
tensor(-82.7738, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-41.0896, device='cuda:0', grad_fn=<NegBackward0>), tensor(-40.6105, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.9264, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-83.3259, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-41.5614, device='cuda:0', grad_fn=<NegBackward0>), tensor(-40.6721, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.9811, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 414
1.1696317776570397
1
This data is wrong to this model, wrong distribution,pass
epoch 415
1.2574136263017697
1
This data is wrong to this model, wrong distribution,pass
epoch 416
1.2465523508378231
1
This data is wrong to this model, wrong distribution,pass
epoch 417
1.0446216141571663
1
This data is wrong to this model, wrong distribution,pass
epoch 418
1.97310126072964
2
This data is wrong to this model, wrong distribution,pass
epoch 419
0.27191797070119733
0
tensor(-83.5780, device='cuda:0',

tensor(-84.8384, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-42.2338, device='cuda:0', grad_fn=<NegBackward0>), tensor(-41.6859, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.5059, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 450
0.283346799456532
0
tensor(-84.7549, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-42.2338, device='cuda:0', grad_fn=<NegBackward0>), tensor(-41.6504, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.3886, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-84.6265, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-42.1955, device='cuda:0', grad_fn=<NegBackward0>), tensor(-41.5761, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.3512, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 451
2.065532806979606
2
This data is wrong to this model, wrong distribution,pass
epoch 452
1.391590068033797
1
This data is wrong to this model, wrong distribution,pass
epoch 453
1.1916074046205571
1
This data is wrong to this model, wrong distribution,pass


epoch 482
0.29129793196496684
0
tensor(-86.3157, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-43.0089, device='cuda:0', grad_fn=<NegBackward0>), tensor(-42.4363, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.3881, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-86.5783, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-43.1287, device='cuda:0', grad_fn=<NegBackward0>), tensor(-42.5861, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.3714, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 483
1.2985878601381922
1
This data is wrong to this model, wrong distribution,pass
epoch 484
1.9742356670923136
2
This data is wrong to this model, wrong distribution,pass
epoch 485
2.117149060372444
2
This data is wrong to this model, wrong distribution,pass
epoch 486
1.3799821703259139
1
This data is wrong to this model, wrong distribution,pass
epoch 487
1.2806986852643183
1
This data is wrong to this model, wrong distribution,pass
epoch 488
1.4176102824733685
1
This data is wrong to t

In [10]:
TT,TF,FT,FF

(165, 0, 0, 315)

In [11]:
torch.save(netD, 'Fashion_MNIST_NetD_changing2-mode11-point7.pth')
torch.save(netV, 'Fashion_MNIST_NetV_changing2-mode11-point7.pth')
torch.save(netG, 'Fashion_MNIST_NetG_changing2-mode11-point7.pth')

In [7]:
# due to storage limits, need to reload and get acc
netD = torch.load('Fashion_MNIST_NetD_changing2-mode11-point7.pth') 
netV = torch.load('Fashion_MNIST_NetV_changing2-mode11-point7.pth') 
netG = torch.load('Fashion_MNIST_NetG_changing2-mode11-point7.pth')

In [8]:
#acc at first for NetD, there exist some issues for netG here, resample will get it right.
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.8329
SVD: 0.8307
NetD, Acc at first 0.8613


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.8279
SVD: 0.8247
NetG, Acc at first 0.8578
